In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


### Shape of data

In [17]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [19]:
np.min(X_train_valid)

-98.828125

In [5]:
np.unique(y_test)

array([769, 770, 771, 772], dtype=int32)

In [7]:
np.unique(person_test)

array([0., 1., 2., 3., 4., 5., 6., 7., 8.])

### A LSTM Example by PyTorch

In [31]:
import torch
rnn = torch.nn.LSTM(input_size=10, hidden_size=20, num_layers=2) 
input = torch.randn(5, 3, 10) # (seq_len, batch, input_size)
#h0 = torch.randn(2, 3, 20) # (num_layers * num_directions, batch, hidden_size)
#c0 = torch.randn(2, 3, 20) # (num_layers * num_directions, batch, hidden_size)
output, (hn, cn) = rnn(input)

In [12]:
hn.size()

torch.Size([2, 3, 20])

### PyTorch Dataloader for EEG data





In [4]:
from torch.utils.data import Dataset, DataLoader

In [5]:
class EEG_Dataset(Dataset):
    def __init__ (self, mode='train'):
        if mode == 'train':
            self.eeg_seq = np.load('X_train_valid.npy')
            label = np.load('y_train_valid.npy')
            self.person_id = np.load('person_train_valid.npy')
        else:
            self.eeg_seq = np.load('X_test.npy')
            label = np.load('y_test.npy')
            self.person_id = np.load('person_test.npy')
        self.label = label - 769 

    def __len__(self):
        return ((self.eeg_seq).shape[0])
    
    def __getitem__(self, idx):
        eeg_seq = torch.from_numpy(self.eeg_seq[idx,:,:]).float()
        label = torch.tensor(self.label[idx]).long()
        person_id = torch.from_numpy(self.person_id[idx,:]).long()     
        sample = {'eeg_seq': eeg_seq, 'label': label, 'person_id': person_id}

        return sample



In [16]:
EEG_trainset = EEG_Dataset(mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_testset = EEG_Dataset(mode='test')
EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=True)

for idx, batch in enumerate(EEG_trainloader):
    print (idx)
    eeg_seq = batch['eeg_seq'].permute(2,0,1)
    print (eeg_seq.size())
    print (batch['label'].size())
    #print (batch['person_id'])



0
torch.Size([1000, 128, 22])
torch.Size([128])
1
torch.Size([1000, 128, 22])
torch.Size([128])
2
torch.Size([1000, 128, 22])
torch.Size([128])
3
torch.Size([1000, 128, 22])
torch.Size([128])
4
torch.Size([1000, 128, 22])
torch.Size([128])
5
torch.Size([1000, 128, 22])
torch.Size([128])
6
torch.Size([1000, 128, 22])
torch.Size([128])
7
torch.Size([1000, 128, 22])
torch.Size([128])
8
torch.Size([1000, 128, 22])
torch.Size([128])
9
torch.Size([1000, 128, 22])
torch.Size([128])
10
torch.Size([1000, 128, 22])
torch.Size([128])
11
torch.Size([1000, 128, 22])
torch.Size([128])
12
torch.Size([1000, 128, 22])
torch.Size([128])
13
torch.Size([1000, 128, 22])
torch.Size([128])
14
torch.Size([1000, 128, 22])
torch.Size([128])
15
torch.Size([1000, 128, 22])
torch.Size([128])
16
torch.Size([1000, 67, 22])
torch.Size([67])


### Use LSTM + FC to perform classification

In [22]:
class model(nn.Module):
    def __init__(self, input_size, hidden_size, class_num):
        super().__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, dropout=0.5)
        self.fc = nn.Linear(hidden_size, class_num)

    def forward(self, x):
        _, (hn, cn) = self.rnn(x)
        out = self.fc(torch.squeeze(hn))
        return out

device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
model = model(input_size=22, hidden_size=100, class_num=4).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

EEG_trainset = EEG_Dataset(mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_testset = EEG_Dataset(mode='test')
EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)

epoch = 20
for i in range (epoch):
    print ('epoch:{}'.format(i+1))
    running_loss = 0.0
    total = 0
    correct = 0
    for idx, batch in enumerate(EEG_trainloader):
        eeg_seq = batch['eeg_seq'].permute(2,0,1).to(device)
        label = batch['label'].to(device)
        optimizer.zero_grad()
        output= model(eeg_seq)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()                
        optimizer.step()
        pred = torch.argmax(output, dim=1)
        correct += torch.sum(pred == label).item()
        total += label.shape[0]
    print ('acc:{}'.format(correct/total))
    print ('loss:{}'.format(running_loss))


epoch:1
acc:0.24397163120567375
loss:23.791213154792786
epoch:2
acc:0.30023640661938533
loss:23.270862936973572
epoch:3
acc:0.3621749408983452
loss:22.97537386417389
epoch:4
acc:0.37446808510638296
loss:22.785637974739075
epoch:5
acc:0.3806146572104019
loss:22.579501390457153
epoch:6
acc:0.4198581560283688
loss:22.32511603832245
epoch:7
acc:0.43215130023640663
loss:22.053317546844482
epoch:8
acc:0.44633569739952716
loss:21.77304756641388
epoch:9
acc:0.4846335697399527
loss:21.460769534111023
epoch:10
acc:0.4879432624113475
loss:21.124024033546448
epoch:11
acc:0.5054373522458628
loss:20.71548843383789
epoch:12
acc:0.5257683215130023
loss:20.330275654792786
epoch:13
acc:0.5361702127659574
loss:19.921437978744507
epoch:14
acc:0.5673758865248227
loss:19.326773524284363
epoch:15
acc:0.6
loss:18.719125032424927
epoch:16
acc:0.6099290780141844
loss:18.1285303235054
epoch:17
acc:0.6368794326241135
loss:17.40804785490036
epoch:18
acc:0.6468085106382979
loss:16.96203923225403
epoch:19
acc:0.6737

In [24]:
running_loss = 0.0
total = 0
correct = 0
for idx, batch in enumerate(EEG_testloader):
    eeg_seq = batch['eeg_seq'].permute(2,0,1).to(device)
    label = batch['label'].to(device)
    output= model(eeg_seq)
    pred = torch.argmax(output, dim=1)
    correct += torch.sum(pred == label).item()
    total += label.shape[0]
print ('acc:{}'.format(correct/total))
#print ('loss:{}'.format(running_loss))

acc:0.25733634311512416
